In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/20201.ai4eng/master/init.py
import init; init.init(force_download=False); init.get_weblink()

In [237]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# `pandas` is mostly about manipulating tables of data

see this cheat sheet: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf


## Pandas main object is a `DataFrame`

- can read .csv, .excel, etc.


In [238]:
!head local/data/internet_facebook.dat

In [239]:
!wc local/data/weather_data_austin_2010.csv

In [240]:
df = pd.read_csv('local/data/internet_facebook.dat', index_col='# Pais')
df

In [241]:
df.head()

In [242]:
df.tail()

In [243]:
df.columns

In [244]:
df.index

**fix the index name**

In [245]:
df.index.name="Pais"
df.head()

In [246]:
df.describe()

In [247]:
df.info()

**a dataframe is made of `Series`**. Observe that each series has **its own type**

In [248]:
s1 = df["Uso_Internet"]
type(s1)

In [249]:
s1

if the column name is not too fancy (empy spaces, accents, etc.) we can use columns names as python syntax.

In [250]:
df.Uso_Facebook

## DataFrame indexing

is **NOT** exactly like numpy

- first index
    - if string refers to columns
    - if `Series` of booleans is used as a filter
    
- for selecting columns:
    - use `.loc` to select by Index
    - use `.iloc` to select by position   

In [251]:
df["Colombia"]

In [252]:
df.loc["Colombia"]

Index semantics is exact!!

In [253]:
df.loc["Colombia":"Spain"]

In [254]:
df.iloc[10:15]

filtering

In [255]:
df[df.Uso_Internet>80]

combined conditions

In [256]:
df[(df.Uso_Internet>50)&(df.Uso_Facebook>50)]

In [257]:
df[(df.Uso_Internet>50)|(df.Uso_Facebook>50)]

## Managing data

    
    
observe csv structure:
- missing column name
- missing data 

In [115]:
!head local/data/comptagevelo2009.csv

In [116]:
d = pd.read_csv("local/data/comptagevelo2009.csv")
d

In [117]:
d.columns, d.shape


numerical features

In [119]:
d.describe()

In [120]:
d["Berri1"].head()

In [121]:
d["Unnamed: 1"].unique()


In [122]:
d["Berri1"].unique()


In [123]:
d["Berri1"].dtype, d["Date"].dtype, d["Unnamed: 1"].dtype


In [124]:
d.index


## Fixing data

observe we set one column as the index one, and we **convert** it to date object type

In [128]:
d.Date

In [129]:
d.index = pd.to_datetime(d.Date)
del(d["Date"])
del(d["Unnamed: 1"])
d.head()

In [130]:
d.index

let's fix columns names

In [131]:
d.columns=["Berri", "Mneuve1", "Mneuve2", "Brebeuf"]
d.head()

In [132]:
for col in d.columns:
    print (col, np.sum(pd.isnull(d[col])))

In [133]:
d.shape

In [134]:
d['Brebeuf'].describe()

In [135]:
plt.hist(d.Brebeuf, bins=30);

**fix missing**!!!

In [139]:
d.Brebeuf.fillna(d.Brebeuf.mean(), inplace=True)


In [140]:
d['Brebeuf'].describe()

In [141]:
plt.hist(d.Brebeuf, bins=30);

In [142]:
d

let's make sure it is sorted

In [144]:
d.sort_index(inplace=True)
d.head()

## Filtering

In [145]:
d[d.Berri>6000]


In [146]:
d[(d.Berri>6000) & (d.Brebeuf<7000)]


## Locating

In [147]:
d[d.Berri>5500].sort_index(axis=0)


In [148]:
d.iloc[100:110]


**dates as INDEX have special semantics**

In [149]:
d.loc["2009-10-01":"2009-10-10"]


can do sorting across any criteria

In [150]:
d.sort_values(by="Berri").head()


and chain operations

In [151]:
d.sort_values(by="Berri").loc["2009-10-01":"2009-10-10"]


# Time series operations

In [152]:
d.rolling(3).mean().head(10)


In [153]:
d.index = d.index + pd.Timedelta("5m")
d.head()

In [154]:
d.shift(freq=pd.Timedelta(days=365)).head()


## Downsampling

In [155]:
d.resample(pd.Timedelta("2d")).first().head()


In [156]:
d.resample(pd.Timedelta("2d")).mean().head()


## Upsampling

In [157]:
d.resample(pd.Timedelta("12h")).first().head()


In [158]:
d.resample(pd.Timedelta("12h")).fillna(method="pad").head()


## Building Dataframes from other structures

In [160]:

a = np.random.randint(10,size=(20,5))
a

In [161]:
k = pd.DataFrame(a, columns=["uno", "dos", "tres", "cuatro", "cinco"], index=range(10,10+len(a)))
k

## `.values` access the underlying `numpy` structure

In [162]:
d.values

## some out-of-the-box plotting

but recall that we always can do custom plotting

In [163]:
d.plot(figsize=(15,3))


In [165]:
plt.figure(figsize=(15,3))
plt.plot(d.Berri)

In [166]:
d.Berri.cumsum().plot()


In [167]:
plt.scatter(d.Berri, d.Brebeuf)


In [168]:
pd.plotting.scatter_matrix(d, figsize=(10,10));

## Grouping

In [169]:

d["month"] = [i.month for i in d.index]
d.head()

In [170]:
d.groupby("month").max()


In [171]:
d.groupby("month").count()


## Time series

observe we can **establish at load time** many thing if the dataset is relatively clean

In [179]:

tiempo=pd.read_csv('local/data/weather_data_austin_2010.csv',parse_dates=['Date'], dayfirst=True ,index_col='Date')
tiempo

In [178]:
tiempo.loc['2010-08-01':'2010-10-30']


In [181]:
tiempo.loc['2010-06'].head()


In [182]:
tiempo.sample(10)


In [183]:
tiempo.sample(frac=0.01)


## Resampling

In [184]:
tiempo.head()


In [185]:
tiempo.resample("5d").mean().head()

In [186]:
tiempo.resample("5d").mean().head()


In [187]:
tiempo.resample("5d").mean().head()


In [191]:
tiempo.resample("30min").mean()[:15]


In [192]:

subt=tiempo.between_time(start_time='1:00',end_time='12:00')
subt

In [193]:
tiempo.index.weekday


In [195]:
tiempo.index.month


In [196]:
tiempo.index.day


In [197]:
tiempo.plot(style='.')


In [201]:
tiempo['2010-01'].plot()

In [202]:
tiempo['2010-01-04'].plot()

## Rolling operations

In [215]:
import pandas as pd
### permite obtener data frames directamente de internet
!pip install yfinance

In [217]:
import yfinance as yf


In [220]:
#define the ticker symbol
tickerSymbol = 'MSFT'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
gs = tickerData.history(period='1d', start='2010-1-1', end='2020-1-25')

#see your data
gs

In [224]:
gs.Close.rolling(10).mean().head(20)


In [232]:
plt.figure(figsize=(20,3))
plt.plot(gs.Close)
plt.plot(gs.Close.rolling(50).mean())


In [233]:
plt.figure(figsize=(20,3))
plt.plot(gs.iloc[:400].Close, label="original")
plt.plot(gs.iloc[:400].Close.rolling(50).mean(), label="rolling")
plt.plot(gs.iloc[:400].Close.rolling(50, center=True).mean(), label="center")
plt.legend();

In [236]:
plt.figure(figsize=(20,3))
plt.plot(gs.iloc[:400].Close.rolling(10).mean())
